# Finetune ruGPT3Medium on essays




## Install enviroment

In [ ]:
!pip3 install folium==0.2.1
!pip3 install urllib3==1.25.4
!pip3 install --no-cache-dir transformers==2.8.0
!pip3 install tqdm
!git clone https://github.com/NVIDIA/apex
!pip3 install -v --no-cache-dir apex

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [ ]:
data_path = "essays.txt"

In [ ]:
!wget https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/history_test.json
!wget https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/russian_test.json
!wget https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/literature_test.json
!wget https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/social_test.json

--2020-11-21 10:36:42--  https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/history_test.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘history_test.json’

history_test.json       [ <=>                ] 107.56K  --.-KB/s    in 0.006s  

2020-11-21 10:36:42 (16.5 MB/s) - ‘history_test.json’ saved [110140]

--2020-11-21 10:36:42--  https://github.com/sberbank-ai/ruGPT3_essays/blob/main/test/russian_test.json
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘russian_test.json’

russian_test.json       [ <=>                ] 483.43K  --.-KB/s    in 0.02s   

2020-11-21 10:36:42 (20.6 MB/s) - ‘russian_test.json’ saved [495035]

--2020-11-21 10:36:43--  https

## Prepare data

In [ ]:
with open(data_path, "r") as file:
    text = file.read()

In [ ]:
valid_size = 10

Было несколько гипотез о том, как представить данные и как обучать модели. В том числе такая, что нужно делать под каждый предмет свою модель. Но так получается в 4 раза меньше данных на каждую модель, а это плохо, т.к. между предметами +- одинаковая структура эссе + перекликаются аргументы.

In [ ]:
topics = []
all_essays = []

topic_lit = []
essays_lit = []
topic_ist = []
essays_ist = []
topic_soc = []
essays_soc = []
topic_rus = []
essays_rus = []

for line, num in zip(text.split("</s>"), range(len(text.split("</s>")))):
    if "Тема:" in line and "Сочинение:" in line:
        essay_text = line.split("Сочинение:")
        if len(essay_text) == 2:
            topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").replace("\n", " ").strip()
            #print(topic, num)
            for i in range(len(topic)):
                topic = topic.replace("({})".format(i), "")
            while len(topic) > 2000:
                topic_list = topic.split(".")
                topic = ". ".join(topic_list[:len(topic_list) // 2 - 2]) + ". ".join(topic_list[len(topic_list) // 2 + 2:])
            if num < 93:
                topic += " Предмет: литература"
            elif num < 179:
                topic += " Предмет: история"
            elif num < 296:
                topic += " Предмет: обществознание"
            else:
                topic += " Предмет: русский язык"
                
            essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").replace("\n", " ").strip()
            essay_text = f"Сочинение: {essay_text}"
            essay_res = f"<s>{topic}\n{essay_text}</s>"
            all_essays.append(essay_res)
            topics.append(topic)

In [ ]:
len(all_essays)

396

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
unique_topics = list(set(topics))

In [ ]:
len(unique_soc)

110

In [ ]:
valid_topics = []

In [ ]:
for _ in range(valid_size):
    # Use randint for more speed (on big lists it is faster)
    idx = np.random.randint(0, len(unique_topics))
    valid_topics.append(unique_topics[idx])

In [ ]:
train = []
valid = []
for topic, essay in zip(topics, all_essays):
    is_train = True
    if topic in valid_topics:
        is_train = False
    if is_train:
        train.append(essay)
    else:
        valid.append(essay)

In [ ]:
len(train), len(valid)

(386, 10)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python 'pretrain_transformers.py' \
            --output_dir='essays_model' \
            --model_type=gpt2 \
            --model_name_or_path=sberbank-ai/rugpt3medium_based_on_gpt2 \
            --do_train \
            --train_data_file=train.txt \
            --do_eval \
            --eval_data_file=valid.txt \
            --per_gpu_train_batch_size 1 \
            --gradient_accumulation_steps 1 \
            --num_train_epochs 12 \
            --block_size 1024 \
            --save_total_limit 5 \
            --overwrite_output_dir 
            #--fp16 \
            #--fp16_opt_level O2 \

11/20/2020 22:14:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/20/2020 22:14:50 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3medium_based_on_gpt2/config.json from cache at /home/jovyan/.cache/torch/transformers/de532b2dc8525a100f796ff660b6ecc57e6e6b7a6ab5892614ebb8cdfc107fba.a2ab8be3b95d09ab7231f8496a99743ed3656b47fea0396c9b5ebb851dd136c1
11/20/2020 22:14:50 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": null,
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "is_decod

## Check our model

In [ ]:
!python3 'generate_transformers.py' \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --path_json='history_test.json' \
    --length=1500

11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   loading file None
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
11/21/2020 02:02:38 - INFO - transformers.tokenization_utils -   loading file essays_model/tokenizer_config.json
11/21/2020 02:02:38 - INFO - transformers.configuration_utils -  

In [ ]:
!python3 'generate_transformers.py' \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --path_json='social_test.json' \
    --length=1500

11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   loading file None
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
11/21/2020 02:33:59 - INFO - transformers.tokenization_utils -   loading file essays_model/tokenizer_config.json
11/21/2020 02:33:59 - INFO - transformers.configuration_utils -  

In [ ]:
!python3 'generate_transformers.py' \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.9 \
    --path_json='russian_test.json' \
    --length=1500

11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   loading file None
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
11/21/2020 09:29:03 - INFO - transformers.tokenization_utils -   loading file essays_model/tokenizer_config.json
11/21/2020 09:29:03 - INFO - transformers.configuration_utils -  

In [ ]:
!python3 'generate_transformers.py' \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --path_json='literature_test.json' \
    --length=1500

11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   loading file None
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
11/21/2020 05:37:35 - INFO - transformers.tokenization_utils -   loading file essays_model/tokenizer_config.json
11/21/2020 05:37:35 - INFO - transformers.configuration_utils -  